## Approche dimensionnelle

In [145]:
import pandas as pd

df = pd.read_csv("../data/clean/merged_data.csv")
df.head(2)

,id,title,price_raw,price,price_original_raw,price_original,discount_percent,url,image_small,image_large,description,promo_type,scrape_date,brand,category,source_site
0,6235,Hyaluron Activ B3 Serum Concentre Repulpant 30ML,"145,000TND",145.0,"181,000TND",181.0,20.0,https://beautystore.tn/promos/6235-hyaluron-ac...,https://beautystore.tn/27600-home_default/hyal...,https://beautystore.tn/27600-large_default/hya...,Le sérum concentré en acide hyaluronique pur (...,Standard,2025-11-24,Avene,NaN,beautystore
1,6234,"Concentré Anti-Imperfections Cleanance ""COMEDO...","61,900TND",61.9,"77,400TND",77.4,20.0,https://beautystore.tn/promos/6234-concentré-a...,https://beautystore.tn/27594-home_default/conc...,https://beautystore.tn/27594-large_default/con...,Le concentré anti-imperfections Cleanance Come...,Standard,2025-11-24,Avene,NaN,beautystore


In [146]:
df.columns

Index(['id', 'title', 'price_raw', 'price', 'price_original_raw',
       'price_original', 'discount_percent', 'url', 'image_small',
       'image_large', 'description', 'promo_type', 'scrape_date', 'brand',
       'category', 'source_site'],
      dtype='object')

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8910 entries, 0 to 8909
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  8910 non-null   object 
 1   title               8910 non-null   object 
 2   price_raw           8910 non-null   object 
 3   price               8910 non-null   float64
 4   price_original_raw  1658 non-null   object 
 5   price_original      1658 non-null   float64
 6   discount_percent    8910 non-null   float64
 7   url                 8910 non-null   object 
 8   image_small         7539 non-null   object 
 9   image_large         8910 non-null   object 
 10  description         8910 non-null   object 
 11  promo_type          8910 non-null   object 
 12  scrape_date         8910 non-null   object 
 13  brand               6045 non-null   object 
 14  category            5099 non-null   object 
 15  source_site         8910 non-null   object 
dtypes: flo

### La complétude (Completeness)

In [148]:
completeness = df.isnull().mean() * 100
completeness

id                     0.000000
title                  0.000000
price_raw              0.000000
price                  0.000000
price_original_raw    81.391695
price_original        81.391695
discount_percent       0.000000
url                    0.000000
image_small           15.387205
image_large            0.000000
description            0.000000
promo_type             0.000000
scrape_date            0.000000
brand                 32.154882
category              42.772166
source_site            0.000000
dtype: float64

* 'price_original' est le prix original du prduit si il y'a promotion, sinon il est NaN

=> Ce champ est structurellement incomplet, pas qualitativement incomplet
* 'image_small' manquants: Certaines fiches n’ont pas d’image miniature 

=> souvent le site met 'image_large' même quand la miniature manque.
* 'brand' et 'category' manquants et sont très importants pour la suite 

=> On s'interesse à les compléter.


1. Brand
    - Extraire les marques des titres des produits lorsque la marque est manquante.
2. Category
    - Construire une table marque -> catégorie majoritaire pour compléter les catégories manquantes.

In [149]:
df['brand'] = df['brand'].str.lower().str.strip()
df['category'] = df['category'].str.lower().str.strip()

In [150]:
import pandas as pd

brand_list = [b for b in df['brand'].dropna().unique()]

def detect_brand(title):
    for b in brand_list:
        if pd.notna(title) and b.lower() in title.lower():
            return b
    return 'Unknown'

df['brand'] = df['brand'].combine_first(df['title'].apply(detect_brand))

In [151]:
brand_category_map = df.dropna(subset=['category']).groupby('brand')['category'].agg(lambda x: x.mode()[0]).to_dict()

df['category'] = df.apply(
    lambda row: row['category'] if pd.notna(row['category']) else brand_category_map.get(row['brand'], 'Unknown'),
    axis=1
)

In [152]:
unknown_brand = (df['brand'] == 'Unknown').sum()
unknown_category = (df['category'] == 'Unknown').sum()

print(f"Brand Unknown: {unknown_brand}")
print(f"Category Unknown: {unknown_category}")

Brand Unknown: 1797
Category Unknown: 2596


3. Price Original
    - Compléter les prix originaux manquants en utilisant le prix actuel et le pourcentage de réduction.

In [153]:
mask = df['price_original'].isna() & (df['discount_percent'] > 0)
df.loc[mask, 'price_original'] = df.loc[mask, 'price']

In [154]:
df['price_original'] = df['price_original'].fillna(df.apply(
    lambda row: row['price'] if row['discount_percent'] == 0 else row['price_original'], axis=1
))

In [155]:
completeness = df.isnull().mean() * 100
completeness

id                     0.000000
title                  0.000000
price_raw              0.000000
price                  0.000000
price_original_raw    81.391695
price_original         0.000000
discount_percent       0.000000
url                    0.000000
image_small           15.387205
image_large            0.000000
description            0.000000
promo_type             0.000000
scrape_date            0.000000
brand                  0.000000
category               0.000000
source_site            0.000000
dtype: float64

Notre dataset est complet.

### La précision (Accuracy)

In [156]:
df[(df['price'] < 0) | (df['price_original'] < 0)].sum()

id                      0
title                   0
price_raw               0
price                 0.0
price_original_raw      0
price_original        0.0
discount_percent      0.0
url                     0
image_small             0
image_large             0
description             0
promo_type              0
scrape_date             0
brand                   0
category                0
source_site             0
dtype: object

In [157]:
df[df['discount_percent'] > 0].query('price_original < price').sum()

id                      0
title                   0
price_raw               0
price                 0.0
price_original_raw      0
price_original        0.0
discount_percent      0.0
url                     0
image_small             0
image_large             0
description             0
promo_type              0
scrape_date             0
brand                   0
category                0
source_site             0
dtype: object

In [158]:
df[df['title'].str.lower().isin(['n/a','unknown','test'])].sum()

id                      0
title                   0
price_raw               0
price                 0.0
price_original_raw      0
price_original        0.0
discount_percent      0.0
url                     0
image_small             0
image_large             0
description             0
promo_type              0
scrape_date             0
brand                   0
category                0
source_site             0
dtype: object

In [159]:
df[~df['url'].str.startswith('http')].sum()

id                      0
title                   0
price_raw               0
price                 0.0
price_original_raw      0
price_original        0.0
discount_percent      0.0
url                     0
image_small             0
image_large             0
description             0
promo_type              0
scrape_date             0
brand                   0
category                0
source_site             0
dtype: object

In [160]:
len(df['brand'].dropna().unique())

266

### La conformité (Validity)

In [161]:
print(df.dtypes)

id                     object
title                  object
price_raw              object
price                 float64
price_original_raw     object
price_original        float64
discount_percent      float64
url                    object
image_small            object
image_large            object
description            object
promo_type             object
scrape_date            object
brand                  object
category               object
source_site            object
dtype: object


In [162]:
print((df['price'] >= 0).all())
print((df['discount_percent'] >= 0).all())

True
True


##### Vérification des formats spécifiques

In [163]:
invalid_urls = df[~df['url'].str.startswith(('http://','https://'))]
print(len(invalid_urls))

0


In [164]:
df['scrape_date_parsed'] = pd.to_datetime(df['scrape_date'], errors='coerce')
invalid_dates = df[df['scrape_date_parsed'].isna()]
print(len(invalid_dates))

0


In [165]:
invalid_discount = df[(df['discount_percent'] > 0) & (df['price_original'].isna())]
print(len(invalid_discount))

0


##### Vérification des règles métier

### La cohérence (Consistency)

* discount_percent = ((price_original - price)/price_original)*100

In [166]:
mismatches_count = ((df['discount_percent'].round(2)) != (((df['price_original'] - df['price']) / df['price_original'] * 100).round(2))).sum()
print(mismatches_count)

1034


In [167]:
df['discount_percent'] = ((df['price_original'] - df['price']) / df['price_original'] * 100).round(2)

* price_original >= price si discount_percent > 0

In [168]:
invalid_price_logic = df[(df['discount_percent'] > 0) & (df['price_original'] < df['price'])]
print(len(invalid_price_logic))

0


### L’actualité (Timeliness)

In [169]:
from datetime import datetime, timedelta
df['scrape_date'] = pd.to_datetime(df['scrape_date'], errors='coerce')
today = pd.to_datetime('today')
max_age_days = 30
stale_data = df[df['scrape_date'] < (today - pd.Timedelta(days=max_age_days))]

print(f"Nombre de lignes avec données trop anciennes (> {max_age_days} jours) : {len(stale_data)}")

Nombre de lignes avec données trop anciennes (> 30 jours) : 0


### La pertinence 

In [170]:
df.columns

Index(['id', 'title', 'price_raw', 'price', 'price_original_raw',
       'price_original', 'discount_percent', 'url', 'image_small',
       'image_large', 'description', 'promo_type', 'scrape_date', 'brand',
       'category', 'source_site', 'scrape_date_parsed'],
      dtype='object')

In [171]:
cols_to_keep = [
    'id',
    'title',
    'price',
    'price_original',
    'discount_percent',
    'promo_type',
    'scrape_date',
    'brand',
    'category',
    'url',
    'image_small',
    'image_large',
    'source_site'
]

df = df[cols_to_keep].copy()

### L’unicité (Uniqueness)

In [172]:
print(f"Lignes totales: {len(df)}")
print(f"Lignes uniques: {df.drop_duplicates().shape[0]}")
print(f"Doublons: {len(df) - df.drop_duplicates().shape[0]}")
print(f"Taux d'unicité: {(df.drop_duplicates().shape[0] / len(df)) * 100:.2f}%")

Lignes totales: 8910
Lignes uniques: 7969
Doublons: 941
Taux d'unicité: 89.44%


In [173]:
df = df.drop_duplicates().reset_index(drop=True)

In [174]:
print(f"Lignes totales: {len(df)}")
print(f"Lignes uniques: {df.drop_duplicates().shape[0]}")
print(f"Doublons: {len(df) - df.drop_duplicates().shape[0]}")
print(f"Taux d'unicité: {(df.drop_duplicates().shape[0] / len(df)) * 100:.2f}%")

Lignes totales: 7969
Lignes uniques: 7969
Doublons: 0
Taux d'unicité: 100.00%


### La clarté (Clarity)

In [175]:
df = df.rename(columns={
    'id': 'product_id',
    'title': 'product_name',
    'price': 'current_price',
    'price_original': 'original_price',
    'discount_percent': 'discount_percent',
    'url': 'product_url',
    'image_small': 'image_thumb',
    'image_large': 'image_full',
    'description': 'product_description',
    'promo_type': 'promotion_type',
    'scrape_date_parsed': 'scrape_date',
    'brand': 'brand_name',
    'category': 'category_name',
    'source_site': 'source_site'
})

In [176]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7969 entries, 0 to 7968
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   product_id        7969 non-null   object        
 1   product_name      7969 non-null   object        
 2   current_price     7969 non-null   float64       
 3   original_price    7969 non-null   float64       
 4   discount_percent  7968 non-null   float64       
 5   promotion_type    7969 non-null   object        
 6   scrape_date       7969 non-null   datetime64[ns]
 7   brand_name        7969 non-null   object        
 8   category_name     7969 non-null   object        
 9   product_url       7969 non-null   object        
 10  image_thumb       6598 non-null   object        
 11  image_full        7969 non-null   object        
 12  source_site       7969 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(9)
memory usage: 809.5+ KB


,product_id,product_name,current_price,original_price,discount_percent,promotion_type,scrape_date,brand_name,category_name,product_url,image_thumb,image_full,source_site
0,6235,Hyaluron Activ B3 Serum Concentre Repulpant 30ML,145.0,181.00,19.89,Standard,2025-11-24,avene,Unknown,https://beautystore.tn/promos/6235-hyaluron-ac...,https://beautystore.tn/27600-home_default/hyal...,https://beautystore.tn/27600-large_default/hya...,beautystore
1,6234,"Concentré Anti-Imperfections Cleanance ""COMEDO...",61.9,77.40,20.03,Standard,2025-11-24,avene,Unknown,https://beautystore.tn/promos/6234-concentré-a...,https://beautystore.tn/27594-home_default/conc...,https://beautystore.tn/27594-large_default/con...,beautystore
2,6233,"Spray Asséchant Réparateur ""Cicalfate+"" 100ml",55.9,69.90,20.03,Standard,2025-11-24,avene,Unknown,https://beautystore.tn/promos/6233-spray-asséc...,https://beautystore.tn/27591-home_default/spra...,https://beautystore.tn/27591-large_default/spr...,beautystore
3,6232,"Crème Réparatrice Multi-protectrice ""Cicalfate...",55.0,68.75,20.00,Standard,2025-11-24,avene,Unknown,https://beautystore.tn/promos/6232-crème-répar...,https://beautystore.tn/27588-home_default/crèm...,https://beautystore.tn/27588-large_default/crè...,beautystore
4,6231,Gelée Gommante Douceur 75ml,38.9,48.60,19.96,Standard,2025-11-24,avene,Unknown,https://beautystore.tn/promos/6231-gelée-gomma...,https://beautystore.tn/27585-home_default/gelé...,https://beautystore.tn/27585-large_default/gel...,beautystore


## Transformation des données

In [179]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

num_cols = ['current_price', 'original_price', 'discount_percent']

# Min-Max Normalization
minmax_scaler = MinMaxScaler()
df[[col + '_norm' for col in num_cols]] = minmax_scaler.fit_transform(df[num_cols])

# Standardization (Z-score)
std_scaler = StandardScaler()
df[[col + '_std' for col in num_cols]] = std_scaler.fit_transform(df[num_cols])

## Réduction des données
### Réduction de la numérosité